In [2]:

import numpy as np
import seaborn as sns
import pandas as pd

<font size='5' face='Times new Roman'>
De US Factors heten: Anomalies_US  
<br>
De Global Factors heten: Anomalies_GF
<br>
De table 1: Anomaliescombinedtable
    <font>

<font size='4' face='Times new Roman'>
    Dit worden de US Factors
    <font>

In [4]:
file_url = 'https://raw.githubusercontent.com/Sebasleen/Seminargroup/Seminar/US%20Factors/anomalies.dta'

# Read the Stata file content
Anomalies = pd.read_stata(file_url)

# Display unique values in the 'anomaly' column
print(Anomalies['anomaly'].unique())

#Hier heb ik de global factors verwijderd
column_name = 'anomaly'
values_to_dropUS = ['glbab', 'glcma', 'glhml', 'glqmj', 'glrmw', 'glsmb', 'glumd']
ElementsUS = Anomalies[column_name].isin(values_to_dropUS)
Anomalies_US = Anomalies[~ElementsUS]

column_name = 'anomaly'
values_to_dropGF = ['ac', 'bab', 'cfp', 'cma', 'ep', 'hml', 'liq', 'ltrev', 'nsi', 'qmj', 'rmw', 'rvar',
'smb', 'strev', 'umd']
ElementsGF = Anomalies[column_name].isin(values_to_dropGF)
Anomalies_GF = Anomalies[~ElementsGF]

#Dit zijn dus alle 15 factors die alleen bij de US horen. 
print(Anomalies_US)

#En dit zijn alle 7 global factors
print(Anomalies_GF)



['ac' 'bab' 'cfp' 'cma' 'ep' 'hml' 'liq' 'ltrev' 'nsi' 'qmj' 'rmw' 'rvar'
 'smb' 'strev' 'umd' 'glbab' 'glcma' 'glhml' 'glqmj' 'glrmw' 'glsmb'
 'glumd']
       year  month anomaly    ret   time  global
0      1963      7      ac  2.170   42.0     0.0
1      1963      8      ac -0.197   43.0     0.0
2      1963      9      ac  0.600   44.0     0.0
3      1963     10      ac  6.463   45.0     0.0
4      1963     11      ac -2.260   46.0     0.0
...     ...    ...     ...    ...    ...     ...
10111  2019      8     umd  7.600  715.0     0.0
10112  2019      9     umd -6.850  716.0     0.0
10113  2019     10     umd  0.240  717.0     0.0
10114  2019     11     umd -2.620  718.0     0.0
10115  2019     12     umd -2.130  719.0     0.0

[10116 rows x 6 columns]
       year  month anomaly       ret   time  global
10116  1987      2   glbab  2.236918  325.0     1.0
10117  1987      3   glbab  1.828450  326.0     1.0
10118  1987      4   glbab -5.521739  327.0     1.0
10119  1987      5   glba

<font size='4' face='Times new Roman'>    

Voor de duidelijkheid, dit zijn de US factors:

- **-ac**: Accruals
- **-bab**: Betting against Beta
- **-cfp**: Cash-flow to price
- **-cma**: Investment (Conservative minus aggressive)
- **-ep**: Earnings to price
- **-hml**: Value
- **-liq**: Liquidity
- **-ltrev**: Long-term reversal
- **-nsi**: Net share issue
- **-qmj**: Quality minus junk
- **-rmw**: Profitability
- **-rvar**: Residual Variance
- **-smb**: Size
- **-strev**: Short-term reversals
- **-umd**: Momentum

En dit zijn de global factors die ik zojuist heb gedropt:

- **glbab**: Betting against beta
- **glcma**: Investment (Conservative minus aggressive)
- **glhml**: Value (high minus low)
- **glqmj**: Quality minus junk
- **glrmw**: Profitability (robust minus weak)
- **glumd**: Momentum (up minus down)
- **glsmb**: Size (small minus big)

</font>


In [228]:
#Table 1: Descriptive Statistics US Factors

AnomaliesUS = Anomalies_US.groupby(['anomaly']).agg({'ret': ['mean', 'std', 'count']}).reset_index()
AnomaliesUS = Anomalies_US.pivot_table(index='anomaly', values='ret', aggfunc=['mean', 'std', 'count'])
AnomaliesUS.columns = ['Mean', 'SD', 'ret_number']
AnomaliesUS.reset_index(inplace=True)

#Calculate the table stuff
AnomaliesUS.columns = ['anomaly', 'Mean', 'SD', 'ret_number']

# Calculate additional statistics
AnomaliesUS['ret_semean'] = AnomaliesUS['SD'] / np.sqrt(AnomaliesUS['ret_number'])

#Times 12 to annualize it
AnomaliesUS['ret'] = AnomaliesUS['Mean'] * 12 

#Times Squareroot 12 to annualize it
AnomaliesUS['sd'] = AnomaliesUS['SD'] * np.sqrt(12) 

#Calculate the t-stat by dividing the return by the standard error of the mean. Divide by 12 to
#annualize it
AnomaliesUS['tstat'] = AnomaliesUS['ret'] / AnomaliesUS['ret_semean'] /12

#Format Table
AnomaliesUS[['Mean']] = AnomaliesUS[['ret']].apply(lambda x: x.map("{:.1f}%".format))
AnomaliesUS[['SD']] = AnomaliesUS[['sd']].apply(lambda x: x.map("{:.1f}%".format))
AnomaliesUS[['T-value']] = AnomaliesUS[['tstat']].apply(lambda x: x.map("{:.2f}".format))

print(AnomaliesUS[['anomaly','Mean','SD','T-value']])


   anomaly  Mean     SD T-value
0       ac  2.8%   6.6%    3.19
1      bab  9.8%  11.2%    6.55
2      cfp  3.4%   8.6%    2.94
3      cma  3.3%   6.9%    3.59
4       ep  3.5%   8.9%    2.95
5      hml  3.6%   9.7%    2.82
6      liq  4.4%  11.6%    2.77
7    ltrev  2.5%   8.7%    2.16
8      nsi  2.8%   8.2%    2.52
9      qmj  4.6%   7.7%    4.47
10     rmw  3.1%   7.5%    3.13
11    rvar  1.6%  17.3%    0.68
12     smb  2.7%  10.4%    1.97
13   strev  6.0%  10.6%    4.21
14     umd  7.8%  14.5%    4.02


In [240]:
#Table 1: Descriptive Statistics US Factors

AnomaliesGF = Anomalies_GF.groupby(['anomaly']).agg({'ret': ['mean', 'std', 'count']}).reset_index()
AnomaliesGF = Anomalies_GF.pivot_table(index='anomaly', values='ret', aggfunc=['mean', 'std', 'count'])
AnomaliesGF.columns = ['Mean', 'SD', 'ret_number']
AnomaliesGF.reset_index(inplace=True)

#Calculate the table stuff
AnomaliesGF.columns = ['anomaly', 'Mean', 'SD', 'ret_number']

# Calculate additional statistics
AnomaliesGF['ret_semean'] = AnomaliesGF['SD'] / np.sqrt(AnomaliesGF['ret_number'])

#Times 12 to annualize it
AnomaliesGF['ret'] = AnomaliesGF['Mean'] * 12 

#Times Squareroot 12 to annualize it
AnomaliesGF['sd'] = AnomaliesGF['SD'] * np.sqrt(12) 

#Calculate the t-stat by dividing the return by the standard error of the mean. Divide by 12 to
#annualize it
AnomaliesGF['tstat'] = AnomaliesGF['ret'] / AnomaliesGF['ret_semean'] /12

#Format Table
AnomaliesGF[['Mean']] = AnomaliesGF[['ret']].apply(lambda x: x.map("{:.1f}%".format))
AnomaliesGF[['SD']] = AnomaliesGF[['sd']].apply(lambda x: x.map("{:.1f}%".format))
AnomaliesGF[['T-value']] = AnomaliesGF[['tstat']].apply(lambda x: x.map("{:.2f}".format))

print(AnomaliesGF[['anomaly','Mean','SD','T-value']])


  anomaly  Mean     SD T-value
0   glbab  9.6%   9.7%    5.70
1   glcma  1.9%   6.0%    1.74
2   glhml  4.0%   7.4%    2.92
3   glqmj  6.2%   6.8%    5.06
4   glrmw  4.3%   4.7%    4.91
5   glsmb  1.1%   7.1%    0.83
6   glumd  7.9%  12.1%    3.54


In [256]:
#Dit zijn beide US en Global factors combined

AnomaliesUScomb = AnomaliesUS[['anomaly', 'Mean', 'SD', 'T-value']]
AnomaliesGFcomb = AnomaliesGF[['anomaly', 'Mean', 'SD', 'T-value']]

Anomaliescombinedtable = pd.concat([AnomaliesUScomb, AnomaliesGFcomb], ignore_index=True)
print(Anomaliescombinedtable)


   anomaly  Mean     SD T-value
0       ac  2.8%   6.6%    3.19
1      bab  9.8%  11.2%    6.55
2      cfp  3.4%   8.6%    2.94
3      cma  3.3%   6.9%    3.59
4       ep  3.5%   8.9%    2.95
5      hml  3.6%   9.7%    2.82
6      liq  4.4%  11.6%    2.77
7    ltrev  2.5%   8.7%    2.16
8      nsi  2.8%   8.2%    2.52
9      qmj  4.6%   7.7%    4.47
10     rmw  3.1%   7.5%    3.13
11    rvar  1.6%  17.3%    0.68
12     smb  2.7%  10.4%    1.97
13   strev  6.0%  10.6%    4.21
14     umd  7.8%  14.5%    4.02
15   glbab  9.6%   9.7%    5.70
16   glcma  1.9%   6.0%    1.74
17   glhml  4.0%   7.4%    2.92
18   glqmj  6.2%   6.8%    5.06
19   glrmw  4.3%   4.7%    4.91
20   glsmb  1.1%   7.1%    0.83
21   glumd  7.9%  12.1%    3.54
